In [1]:
import os
import pandas as pd
import datetime
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [2]:
df = pd.DataFrame()
for f in [x for x in os.listdir("data") if x[2] == "_"]:
    df = pd.concat([df, pd.read_parquet(os.path.join("data",f))])

In [3]:
df['oscrapovano_minuty'] = df['oscrapovano'].apply(lambda x: str(x)[0:14])

In [4]:
df['oscrapovano_minuty']

0       2024-11-07 18:
1       2024-11-07 18:
2       2024-11-07 18:
3       2024-11-07 18:
4       2024-11-07 18:
             ...      
1601    2024-12-01 23:
1602    2024-12-01 23:
1603    2024-12-01 23:
1604    2024-12-01 23:
1605    2024-12-01 23:
Name: oscrapovano_minuty, Length: 2162192, dtype: object

In [5]:
df = df.drop_duplicates(subset=['odkud','kam','odjezd','oscrapovano_minuty'], keep="last")

In [6]:
len(df)

1027884

In [7]:
df[['odjezd','odkud','kam']].drop_duplicates().shape[0]

77102

In [8]:
df.groupby(['prodejce',pd.Grouper(key='oscrapovano',freq='D')]).size()

prodejce  oscrapovano
ARR       2024-11-07      2297
          2024-11-08      7128
          2024-11-09      3320
          2024-11-10      3157
          2024-11-11      3367
          2024-11-12      3323
          2024-11-13      3632
          2024-11-14      3072
          2024-11-15      3552
          2024-11-16      3498
          2024-11-17      3254
          2024-11-18      3310
          2024-11-19      3384
          2024-11-20      3393
          2024-11-21      3356
          2024-11-22      3589
          2024-11-23      3496
          2024-11-24      3394
          2024-11-25      3086
          2024-11-26      2799
          2024-11-27      2986
          2024-11-28      3206
          2024-11-29      3036
          2024-11-30      3005
          2024-12-01      3048
LE        2024-11-07       332
          2024-11-08      1573
          2024-11-09      1873
          2024-11-10      1755
          2024-11-11      1697
          2024-11-12      1889
          2024-11

Výpadek ČD 2024-11-26 opraven, vyřešen.

In [10]:
df = df.sort_values(by="oscrapovano").reset_index(drop=True)

In [11]:
df = df.dropna(subset=['odkud','kam','odjezd','oscrapovano'],how='any')

In [12]:
df = df[df['prostredek'] != 'autobus']

In [13]:
df = df[~df['kam'].str.contains('(tief)',na=False)]
df = df[~df['odkud'].str.contains('(tief)',na=False)]

C:\Users\micha\AppData\Local\Temp\ipykernel_25384\103445691.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df['kam'].str.contains('(tief)',na=False)]
C:\Users\micha\AppData\Local\Temp\ipykernel_25384\103445691.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df['odkud'].str.contains('(tief)',na=False)]


Před 7. listopadem scrapování testovací a nesystematické = nevypovídající = filtrujeme.

In [15]:
df = df[df['oscrapovano'] >= '2024-11-08']

In [16]:
df.groupby(['prodejce',pd.Grouper(key='oscrapovano',freq='D')]).size()

prodejce  oscrapovano
ARR       2024-11-08      7128
          2024-11-09      3320
          2024-11-10      3157
          2024-11-11      3367
          2024-11-12      3323
          2024-11-13      3632
          2024-11-14      3072
          2024-11-15      3552
          2024-11-16      3498
          2024-11-17      3254
          2024-11-18      3310
          2024-11-19      3384
          2024-11-20      3393
          2024-11-21      3356
          2024-11-22      3589
          2024-11-23      3496
          2024-11-24      3394
          2024-11-25      3086
          2024-11-26      2799
          2024-11-27      2986
          2024-11-28      3206
          2024-11-29      3036
          2024-11-30      3005
          2024-12-01      3048
LE        2024-11-08      1573
          2024-11-09      1873
          2024-11-10      1755
          2024-11-11      1697
          2024-11-12      1889
          2024-11-13      1886
          2024-11-14      1981
          2024-11

In [17]:
days = {0: 'po', 1: 'út', 2: 'st', 3: 'čt', 
        4: 'pá', 5: 'so', 6: 'ne'}
df['den'] = df['odjezd'].dt.dayofweek.map(days)

In [18]:
df['predstih_d'] = df['predstih'].dt.days
df['predstih_h'] = df['predstih'].dt.total_seconds() / 3600

In [19]:
df = df[df['predstih_h'] > -3]

In [20]:
kategoricka_data = ['odkud','kam','prodejce','den']

In [21]:
for k in kategoricka_data:
    print("Before:", df[k].memory_usage(deep=True))
    df[k] = df[k].astype('category')
    print("After: ", df[k].memory_usage(deep=True))

Before: 53447100
After:  6689094
Before: 53576591
After:  6687437
Before: 54881798
After:  6680209
Before: 49126971
After:  6680529


In [22]:
df['cena'] = pd.to_numeric(df['cena'])

In [23]:
df.shape[0]

742201

In [24]:
df[['odjezd','odkud','kam']].drop_duplicates().shape[0]

63799

In [25]:
poradi = ['oscrapovano','prodejce','odkud','kam',
 'odjezd',
 'predstih',
 'predstih_d',
 'predstih_h',
          'cena',
 'prostredek',
          'volnych_mist',
 'obsazenost',
 'jizdni_doba',
 'vzdalenost',
 'zpozdeni',
 'cena_poznamka',
 'den',
 'prestupy',
'vlaky',
 'mistenka_zdarma',
 'nahradni_bus',
 'volna_mista_economy',
 'volna_mista_economy_plus',
 'volna_mista_economy_business',
 'volna_mista_premium']

In [26]:
df[poradi].to_parquet(os.path.join("data","jizdenky.parquet"))

In [27]:
nejnovejsi = df['oscrapovano'].max()
nejnovejsi
df_tyden = df[df['oscrapovano'] > (nejnovejsi - datetime.timedelta(hours=168))]

In [28]:
df_tyden[poradi].to_csv(os.path.join("data","jizdenky_tyden.csv"))